In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# View training data - data to train the model - contains the target value (y) Survived
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [5]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


The code cell below looks for patterns in four different columns ("Pclass", "Sex", "SibSp", and "Parch") of the data. It constructs the trees in the random forest model based on patterns in the train.csv file, before generating predictions for the passengers in test.csv. The code also saves these new predictions in a CSV file submission.csv.


In [6]:
# Importing the RandomForestClassifier from sklearn
from sklearn.ensemble import RandomForestClassifier

# Defining the target variable (y) as the "Survived" column in the training data
y_train = train_data["Survived"]

# Defining the feature columns to be used for prediction
features = ["Pclass", "Sex", "SibSp", "Parch"]

# Using pd.get_dummies to convert categorical features like "Sex" into numerical values (One-Hot Encoding)
X_train = pd.get_dummies(train_data[features])

# Applying the same One-Hot Encoding to the test data
X_test = pd.get_dummies(test_data[features])
print(X_test.head())  # Display the first few rows of the test feature matrix

# Creating a Random Forest Classifier model with 100 trees and a max depth of 5
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

# Training the model on the training data (X, y)
rf.fit(X_train, y_train)

# Using the trained model to make predictions on the test data
y_pred = rf.predict(X_test)

# Creating a DataFrame to store the "PassengerId" from the test set and the predicted "Survived" values
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': y_pred})

# Saving the predictions to a CSV file for submission
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")  # Notify the user that the file is saved


   Pclass  SibSp  Parch  Sex_female  Sex_male
0       3      0      0       False      True
1       3      1      0        True     False
2       2      0      0       False      True
3       3      0      0       False      True
4       3      1      1        True     False
Your submission was successfully saved!


Decision tree classifier


In [7]:
# Import model
from sklearn.tree import DecisionTreeClassifier

# Import train_test_split
from sklearn.model_selection import train_test_split

# Import accuracy metric
from sklearn.metrics import accuracy_score

# Target variable (what we want to predict)
y = train_data["Survived"]

# Feature columns (inputs)
features = ["Pclass", "Sex", "SibSp", "Parch"]

# Convert categorical variables to numeric
X = pd.get_dummies(train_data[features])

# Split the dataset into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    test_size=0.2,
    stratify=y,        
    random_state=1
)

# Instantiate Decision Tree model
dt = DecisionTreeClassifier(max_depth=6, random_state=1)

# Train the model
dt.fit(X_train, y_train)

# Make predictions on the test split
y_pred = dt.predict(X_test)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", accuracy)


Test Accuracy: 0.8044692737430168
